In [33]:
import pandas as pd
import ssl
import numpy as np
import plotly.express as px
from scipy.stats import linregress
import plotly.graph_objects as go

In [34]:
# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

def clean_dataset(df):
    """Cleans the dataset: removes duplicates, handles missing values, sorts, and drops unnecessary columns."""
    df_cleaned = df.drop_duplicates()

    # Handle missing values (NaN)
    if df_cleaned.isnull().any().any():
        # Fill NaN values with the mean of numeric columns
        df_cleaned = df_cleaned.fillna(df_cleaned.mean(numeric_only=True))

    # Sort data by a specific column (e.g., Year)
    if "Year" in df_cleaned.columns:
        df_cleaned = df_cleaned.sort_values(by="Year")

    # Drop irrelevant columns (e.g., ID)
    if "ID" in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=["ID"])

    return df_cleaned

In [35]:
def fetch_data():
    """Fetches datasets from the specified URLs."""
    emissions_data = pd.read_csv(
        "https://ourworldindata.org/grapher/annual-co-emissions-by-region.csv?v=1&csvType=full&useColumnShortNames=true",
        storage_options={'User-Agent': 'Our World In Data data fetch/1.0'}
    )

    temperature_data = pd.read_csv(
        "https://ourworldindata.org/grapher/monthly-average-surface-temperatures-by-year.csv?v=1&csvType=full&useColumnShortNames=false",
        storage_options={'User-Agent': 'Our World In Data data fetch/1.0'}
    )

    return emissions_data, temperature_data

In [36]:
def transform_temperature_data(temperature_data):
    """Transforms the temperature dataset into an analyzable format."""
    temperature_data = clean_dataset(temperature_data)

    # Ensure column names for years are numeric
    temperature_data.columns = [
        int(col) if col.isdigit() else col
        for col in temperature_data.columns
    ]

    # Unpivot temperature data
    temperature_melted = temperature_data.melt(
        id_vars=["Entity", "Code", "Year"],
        var_name="Year_Column",
        value_name="Temperature"
    )

    # Convert 'Year_Column' to numeric values
    temperature_melted["Year_Column"] = pd.to_numeric(temperature_melted["Year_Column"], errors="coerce")

    # Rename columns
    temperature_melted.rename(columns={"Year_Column": "Year", "Year": "Month"}, inplace=True)

    return temperature_melted


In [37]:
def filter_data(temperature_data, emissions_data, regions):
    """Filters the temperature and emissions data for the specified regions and shared years."""

    temperature_filtered = temperature_data[temperature_data["Entity"].isin(regions)]
    emissions_filtered = emissions_data[emissions_data["Entity"].isin(regions)]

    # Find common years
    common_years = set(temperature_filtered["Year"]).intersection(set(emissions_filtered["Year"]))

    temperature_filtered = temperature_filtered[temperature_filtered["Year"].isin(common_years)]
    emissions_filtered = emissions_filtered[emissions_filtered["Year"].isin(common_years)]

    return temperature_filtered, emissions_filtered



In [38]:
def merge_datasets(temperature_data, emissions_data):
    """Merges the temperature and emissions datasets into a single dataset."""
    combined_data = pd.merge(
        temperature_data,
        emissions_data,
        on=["Entity", "Year"],
        how="inner"
    )
    return combined_data

In [39]:
def plot_temp_by_country(emissions_data, region_countries, region_name):
    """Erstellt ein interaktives Liniendiagramm der CO2-Emissionen für alle Länder einer Region."""
    # Daten filtern
    region_data = emissions_data[emissions_data["Entity"].isin(region_countries)]

    # Interaktives Plotly-Diagramm erstellen
    fig = px.line(
        region_data,
        x="Year",
        y="Temperature",
        color="Entity",
        title=f"Temperature in {region_name} nach Ländern",
        labels={"Temperature": "Temperature", "Year": "Jahr", "Entity": "Entity"}
    )

    # Layout anpassen
    fig.update_layout(
        legend_title_text="Länder",
        title_font_size=16,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12
    )

    # Grafik anzeigen
    fig.show()

In [40]:
def calculate_p_values(df_combined):
    """Berechnet die lineare Regression und die p-Werte für jede Region."""
    results = []
    
    for region in df_combined["Region"].unique():
        region_data = df_combined[df_combined["Region"] == region]
        x = region_data["Year"].values
        y = region_data["emissions_total"].values

        # Lineare Regression
        slope, intercept, r_value, p_value, std_err = linregress(x, y)

        # Ergebnisse speichern
        results.append({
            "Region": region,
            "Slope": slope,
            "Intercept": intercept,
            "R-squared": r_value**2,
            "P-value": p_value
        })

    return pd.DataFrame(results)

In [41]:
emissions_data, temperature_data = fetch_data()

# Transform temperature data
temperature_melted = transform_temperature_data(temperature_data)

temperature_melted.head()

,Entity,Code,Month,Year,Temperature
0,Afghanistan,AFG,1,2024,3.300064
1,Rwanda,RWA,1,2024,19.979807
2,Cook Islands,COK,1,2024,25.643768
3,Saint Helena,SHN,1,2024,23.700409
4,Congo,COG,1,2024,26.152718


In [42]:
emissions_data.head()

,Entity,Code,Year,emissions_total
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0


In [43]:
    # Define regions
north_america_countries = [
        "Antigua and Barbuda", "Bahamas", "Belize", "Costa Rica",
        "Dominican Republic", "El Salvador", "Haiti", "Honduras", 
        "Jamaica", "Canada", "Cuba", "Mexico", "Nicaragua", 
        "Panama", "Trinidad and Tobago", "United States"
]

south_america_countries = [
        "Argentina", "Bolivia", "Brazil", "Chile", "Ecuador",
        "Guyana", "Colombia", "Paraguay", "Peru", "Suriname",
        "Uruguay", "Venezuela", "Guatemala"
]

all_americas = north_america_countries + south_america_countries

In [44]:
# Gruppieren nach Jahr und Entity, Berechnung der jährlichen Werte
yearly_summary_n = temperature_melted.groupby(["Entity", "Year"]).agg({
    "Temperature": "mean",
}).reset_index()

# Umbenennen der Spalten für besseres Format
yearly_summary_n.rename(columns={
    "Entity": "Entity",
    "Year": "Year",
    "Temperature": "Temperature"
}, inplace=True)

yearly_summary_n.head()


,Entity,Year,Temperature
0,Afghanistan,1950,10.231125
1,Afghanistan,1951,11.242811
2,Afghanistan,1952,11.292934
3,Afghanistan,1953,11.445144
4,Afghanistan,1954,11.093394


In [45]:
# Filter and merge datasets for North America
yearly_summary_na_filtered, emissions_filtered_na = filter_data(yearly_summary_n, emissions_data, north_america_countries)


In [46]:
combined_filtered_data_na = merge_datasets(yearly_summary_na_filtered, emissions_filtered_na)
combined_filtered_data_na.head()

,Entity,Year,Temperature,Code,emissions_total
0,Antigua and Barbuda,1950,25.377907,ATG,12213.0
1,Antigua and Barbuda,1951,25.850764,ATG,19541.0
2,Antigua and Barbuda,1952,25.956604,ATG,14656.0
3,Antigua and Barbuda,1953,25.972994,ATG,17099.0
4,Antigua and Barbuda,1954,25.597811,ATG,12213.0


In [47]:
# Gruppieren nach Jahr und Entity, Berechnung der jährlichen Werte
yearly_summarynorden = combined_filtered_data_na.groupby( "Year").agg({
    "Temperature": "mean",  # Durchschnittstemperatur über alle Monate
    "emissions_total": "sum"  # Gesamtemissionen über alle Monate
}).reset_index()

yearly_summarynorden.head()

,Year,Temperature,emissions_total
0,1950,20.861395,2.736695e+09
1,1951,21.306945,2.828319e+09
2,1952,21.487919,2.760658e+09
3,1953,21.643270,2.823216e+09
4,1954,21.200929,2.701530e+09


In [48]:
# Filter and merge datasets for North America
yearly_summary_sa_filtered, emissions_filtered_na = filter_data(yearly_summary_n, emissions_data, south_america_countries)

In [49]:
sueden = merge_datasets(yearly_summary_sa_filtered, emissions_filtered_na)
sueden.head()

,Entity,Year,Temperature,Code,emissions_total
0,Argentina,1950,14.214175,ARG,29921192.0
1,Argentina,1951,14.407666,ARG,34962916.0
2,Argentina,1952,14.316195,ARG,36095124.0
3,Argentina,1953,14.520419,ARG,35134740.0
4,Argentina,1954,13.677547,ARG,36750356.0


In [50]:
# Gruppieren nach Jahr und Entity, Berechnung der jährlichen Werte
yearly_summarysouth = sueden.groupby( "Year").agg({
    "Temperature": "mean",  # Durchschnittstemperatur über alle Monate
    "emissions_total": "sum"  # Gesamtemissionen über alle Monate
}).reset_index()

yearly_summarysouth.head()

,Year,Temperature,emissions_total
0,1950,19.933421,114035625.0
1,1951,20.239919,131914233.0
2,1952,20.264052,142109627.0
3,1953,20.444680,141313611.0
4,1954,19.866740,151785007.0


In [51]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_emissions_by_country_large_graph(emissions_data_na, countries_na, region_na, 
                                          emissions_data_sa, countries_sa, region_sa, output_file):
    """Erstellt eine große Grafik für CO2-Emissionen zweier Regionen nebeneinander und speichert sie als Bild."""
    # Daten filtern
    region_data_na = emissions_data_na[emissions_data_na["Entity"].isin(countries_na)]
    region_data_sa = emissions_data_sa[emissions_data_sa["Entity"].isin(countries_sa)]

    # Subplots erstellen
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=(f"CO2-Emissionen in {region_na}", f"CO2-Emissionen in {region_sa}")
    )

    # Nordamerika-Daten hinzufügen
    for country in countries_na:
        country_data = region_data_na[region_data_na["Entity"] == country]
        fig.add_trace(
            go.Scatter(x=country_data["Year"], y=country_data["emissions_total"], mode='lines', name=country),
            row=1, col=1
        )

    # Südamerika-Daten hinzufügen
    for country in countries_sa:
        country_data = region_data_sa[region_data_sa["Entity"] == country]
        fig.add_trace(
            go.Scatter(x=country_data["Year"], y=country_data["emissions_total"], mode='lines', name=country),
            row=1, col=2
        )

    # Layout anpassen
    fig.update_layout(
        title_text="CO2-Emissionen nach Ländern: Nordamerika und Südamerika",
        title_font_size=20,
        showlegend=True,
        legend_title_text="Länder",
        xaxis=dict(title="Jahr"),
        yaxis=dict(title="CO2-Emissionen (Millionen Tonnen)"),
        height=800,  # Höhe der Grafik in Pixel
        width=1600   # Breite der Grafik in Pixel
    )

    # Grafik speichern
    fig.write_image(output_file)
    print(f"Die Grafik wurde erfolgreich als {output_file} gespeichert.")

# Beispielaufruf
plot_emissions_by_country_large_graph(
    combined_filtered_data_na, north_america_countries, "Nordamerika",
    sueden, south_america_countries, "Südamerika",
    "co2_emissions_large_graph.png"
)


Die Grafik wurde erfolgreich als co2_emissions_large_graph.png gespeichert.


In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_temperature_by_region_large_graph(temp_data_na, countries_na, region_na, 
                                           temp_data_sa, countries_sa, region_sa, output_file):
    """Erstellt eine große Grafik für Temperaturen zweier Regionen nebeneinander und speichert sie als Bild."""
    # Daten filtern
    region_data_na = temp_data_na[temp_data_na["Entity"].isin(countries_na)]
    region_data_sa = temp_data_sa[temp_data_sa["Entity"].isin(countries_sa)]

    # Subplots erstellen
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=(f"Temperaturen in {region_na}", f"Temperaturen in {region_sa}")
    )

    # Nordamerika-Daten hinzufügen
    for country in countries_na:
        country_data = region_data_na[region_data_na["Entity"] == country]
        fig.add_trace(
            go.Scatter(x=country_data["Year"], y=country_data["Temperature"], mode='lines', name=country),
            row=1, col=1
        )

    # Südamerika-Daten hinzufügen
    for country in countries_sa:
        country_data = region_data_sa[region_data_sa["Entity"] == country]
        fig.add_trace(
            go.Scatter(x=country_data["Year"], y=country_data["Temperature"], mode='lines', name=country),
            row=1, col=2
        )

    # Layout anpassen
    fig.update_layout(
        title_text="Temperaturen nach Ländern: Nordamerika und Südamerika",
        title_font_size=20,
        showlegend=True,
        legend_title_text="Länder",
        xaxis_title="Jahr",
        yaxis_title="Temperatur (°C)",
        height=800,  # Höhe der Grafik in Pixel
        width=1600   # Breite der Grafik in Pixel
    )

    # Grafik speichern
    fig.write_image(output_file)
    print(f"Die Grafik wurde erfolgreich als {output_file} gespeichert.")

# Beispielaufruf
plot_temperature_by_region_large_graph(
    combined_filtered_data_na, north_america_countries, "Nordamerika",
    sueden, south_america_countries, "Südamerika",
    "temperature_large_graph.png"
)


Die Grafik wurde erfolgreich als temperature_large_graph.png gespeichert.


In [52]:
plot_temp_by_country(combined_filtered_data_na, north_america_countries, "Nordamerika")

In [53]:
plot_temp_by_country(sueden, south_america_countries, "Südamerika")

In [54]:
def plot_temperature_vs_emissions(df_combined):
    """
    Erstellt ein Streudiagramm, das die Temperatur in Abhängigkeit der Emissionen zeigt,
    inklusive einer Regressionslinie.
    """
    # Regressionslinie berechnen
    x = df_combined["emissions_total"]
    y = df_combined["Temperature"]

    coeffs = np.polyfit(x, y, deg=1)  # lineare Regression
    trendline = np.polyval(coeffs, x)

    # Streudiagramm erstellen
    fig = px.scatter(
        df_combined,
        x="emissions_total",
        y="Temperature",
        color="Region",
        title="Temperatur in Abhängigkeit von CO2-Emissionen",
        labels={"Gesamtemissionen": "CO2-Emissionen (Millionen Tonnen)", "Durchschnittstemperatur": "Temperatur (°C)"},
        hover_data=["Year"]
    )

    # Grafik anzeigen
    fig.show()

In [55]:
    # Region hinzufügen, um die Daten zu unterscheiden
yearly_summarynorden["Region"] = "Nordamerika"
yearly_summarysouth["Region"] = "Südamerika"

df_combined = pd.concat([yearly_summarynorden, yearly_summarysouth], ignore_index=True)

plot_temperature_vs_emissions(df_combined)

In [56]:
def plot_temperature_with_trendlines(df_combined, p_values_df):
    """Erstellt eine Plotly-Grafik, die Temperaturdaten mit Trendlinien und P-Werten anzeigt."""
    fig = go.Figure()

    # Durch Regionen iterieren und Temperaturdaten mit Trendlinien hinzufügen
    for region in df_combined["Region"].unique():
        region_data = df_combined[df_combined["Region"] == region]
        x = region_data["Year"]
        y = region_data["Temperature"]

        # Plot Temperaturdaten
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode="lines",
            name=f"{region} Temperature",
            line=dict(width=2),
            hovertemplate="Jahr: %{x}<br>Temperature: %{y:.2f}°C<extra></extra>"
        ))

        # Lineare Regression berechnen
        coeffs = np.polyfit(x, y, deg=4)
        trendline = np.polyval(coeffs, x)

        # Trendlinie hinzufügen
        fig.add_trace(go.Scatter(
            x=x,
            y=trendline,
            mode="lines",
            name=f"{region} Trendlinie",
            line=dict(dash="dash"),
            hovertemplate="Jahr: %{x}<br>Trendlinie: %{y:.2f}°C<extra></extra>"
        ))

    # Layout anpassen
    fig.update_layout(
        title="Temperaturänderungen in Nord- und Südamerika mit Trendlinien",
        xaxis_title="Jahr",
        yaxis_title="Temperature (°C)",
        legend_title="Regionen",
        template="plotly_white"
    )

    # Grafik anzeigen
    fig.show()

In [57]:
def calculate_p_values(df_combined):
    """Berechnet die lineare Regression und die p-Werte für jede Region."""
    results = []
    
    for region in df_combined["Region"].unique():
        region_data = df_combined[df_combined["Region"] == region]
        x = region_data["Year"].values
        y = region_data["emissions_total"].values

        # Lineare Regression
        slope, intercept, r_value, p_value, std_err = linregress(x, y)

        # Ergebnisse speichern
        results.append({
            "Region": region,
            "Slope": slope,
            "Intercept": intercept,
            "R-squared": r_value**2,
            "P-value": p_value
        })

    return pd.DataFrame(results)


In [58]:
# P-Wert-Berechnung
p_values_df = calculate_p_values(df_combined)
print("\nP-Werte und Regressionsergebnisse:")
print(p_values_df)


P-Werte und Regressionsergebnisse:
        Region         Slope     Intercept  R-squared       P-value
0  Nordamerika  5.990640e+07 -1.136031e+11    0.80692  2.011357e-27
1   Südamerika  1.580106e+07 -3.077868e+10    0.96686  5.146013e-55


In [59]:
# Aufruf der Funktion
plot_temperature_with_trendlines(df_combined, p_values_df)